# Import Dependencies

In [1]:
import os
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.test import utils
from gensim.models import KeyedVectors
from nltk import tokenize
from nltk.corpus import stopwords
from nltk import download
import pandas as pd
import smart_open
import numpy as np

C:\Users\aidan\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load Data

In [ ]:
#os.mkdir('word2vec')

In [3]:
os.chdir(r'C:\Users\aidan\OneDrive - University of Bath\1_Semester_2\Cm50175_dissertation_preparation\Data\consolidate\final_data')

In [4]:
abstracts_df = pd.read_csv('abstracts.csv')

before = abstracts_df.shape[0]
abstracts_df.dropna(subset=['description'], inplace=True)
abstracts_df.reset_index(drop=True, inplace=True)
after = abstracts_df.shape[0]

print(f'Dropped {before - after} rows due to missing abstracts.')

Dropped 60267 rows due to missing abstracts.


# Process Data

Unlike in doc2vec, the corpus has to be separated by sentences, not documents.

In [5]:
stop_words = stopwords.words('english')

In [ ]:
class MyCorpus:
    
    def __iter__(self):
        for _, row in abstracts_df.iterrows():
            for line in tokenize.sent_tokenize(row['description']):
                words = utils.simple_preprocess(line)
                words = [word for word in words if word not in stop_words]
                yield words

In [ ]:
sentences = MyCorpus()

# Train Model

In [ ]:
vector_size = 1000
alpha = 0.025
min_alpha = 0.0
min_count = 2
window_size = 5
epochs = 10
workers = 2

In [ ]:
model = Word2Vec(sentences,
                 vector_size=vector_size,
                 alpha=alpha,
                 min_alpha=min_alpha,
                 min_count=min_count,
                 epochs=epochs,
                 workers=workers)

# Save Model

This automatically saves the model and word vectors as well.

NOT ENOUGH MEMORY?? Is it saving the word vectors and not the 

In [ ]:
fname = r'C:\Users\aidan\OneDrive - University of Bath\1_Semester_2\Cm50175_dissertation_preparation\Data\consolidate\embeddings\word2vec'
os.chdir(fname)

# Save keyed vectors! This was annoying to find out
wv = model.wv
wv.save(f'my_w2v_{vector_size}.kv')

# Calculate Document Vectors

In [7]:
os.chdir('..\\embeddings\\word2vec')

In [8]:
class MyDocCorpus:
    
    def __iter__(self):
        for _, row in abstracts_df.iterrows():
            words = utils.simple_preprocess(row['description'])
            words = [word for word in words if word not in stop_words]
            yield words

In [9]:
docs = MyDocCorpus()

In [23]:
def get_doc_embedding(word2vec, doc):
    # Remove OOV words
    doc = [word for word in doc if word2vec.has_index_for(word)]
    return np.mean(word2vec[doc], axis=0)

In [40]:
num_docs = 767968
vector_size = 1000
wv = KeyedVectors.load(f'my_w2v_{vector_size}.kv')
print('Vectors loaded')
not_valid = 0
to_save = np.empty((vector_size, num_docs))

for i, doc in enumerate(docs):
    try:
        to_save[:,i] = get_doc_embedding(wv, doc)
    except Exception as e:
        print(i)
        print(e)
        break

Vectors loaded


MemoryError: Unable to allocate 5.72 GiB for an array with shape (1000, 767968) and data type float64

In [34]:
np.save(f'my_w2v_{vector_size}_docvecs.npy', to_save.T)

In [39]:
%whos

Variable            Type                    Data/Info
-----------------------------------------------------
KeyedVectors        type                    <class 'gensim.models.keyedvectors.KeyedVectors'>
MyDocCorpus         type                    <class '__main__.MyDocCorpus'>
Word2Vec            type                    <class 'gensim.models.word2vec.Word2Vec'>
abstracts_df        DataFrame                       Unnamed: 0       <...>[767968 rows x 4 columns]
after               int                     767968
before              int                     828235
doc                 list                    n=152
docs                MyDocCorpus             <__main__.MyDocCorpus obj<...>ct at 0x0000027C1390B550>
download            method                  <bound method Downloader.<...>t at 0x0000027BAA4724E0>>
get_doc_embedding   function                <function get_doc_embeddi<...>ng at 0x0000027BAC2A41E0>
i                   int                     767967
not_valid           int           